## Purpose of notebook: 

    Annotate and identify problems with incorporating p4zflx.F90 from NEMO-3.6-code/NEMOGCM/NEMO/TOP_SRC/PISCES/P4Z
    into SKOG. below - A) my gutted p4zflx, with changes. B) ungutted but annotated pisces p4zflx
    
2 main (technical) questions:

    -How to initialize hi? should we do an addition to  p4zsms 
    -rhop, rtrn - what are they?
    
one science question:

    -I know where they're getting some of the chem constants, but don't have a ref for what they're doing with the surface chemistry itself. 

    Everything else *should* follow...

# My p4zflx, based on PISCES, with the same annotations as the below code

    SUBROUTINE p4z_flx ( kt, knt, ln_co2sys )
    !!---------------------------------------------------------------------
          !!                     ***  ROUTINE p4z_flx  ***
          !!
          !! ** Purpose :   CALCULATES GAS EXCHANGE AND CHEMISTRY AT SEA SURFACE
          !!
          !! ** Method  :
          !!              - Include total atm P correction via Esbensen & Kushnir (1981)
          !!              - Pressure correction NOT done for key_cpl_carbon_cycle
          !!              - Remove Wanninkhof chemical enhancement;
          !!              - Add option for time-interpolation of atcco2.txt
          !!---------------------------------------------------------------------
          !


          INTEGER, INTENT( in ) ::   kt, knt
          LOGICAL, INTENT( in ) :: ln_co2sys
          INTEGER  ::   ji, jj, jm, iind, iindm1
          REAL(wp) ::   ztc, ztc2, ztc3, ztc4, zws, zkgwan
          REAL(wp) ::   zfld, zflu, zfld16, zflu16, zfact
          REAL(wp) ::   zph, zah2, zbot, zdic, zalk, zsch_o2, zalka, zsch_co2
          REAL(wp) ::   zyr_dec, zdco2dt
          CHARACTER (len=25) :: charout
          REAL(wp), POINTER, DIMENSION(:,:) :: zkgco2, zkgo2 !, zh2co3, zoflx, zw2d

          IF( nn_timing == 1 )  CALL timing_start('p4z_flx')
          !
          CALL wrk_alloc( jpi, jpj, zkgco2, zkgo2) ! , zh2co3, zoflx )

    !~~~~~~~~~~~~walrus F_A need to calculate surface carbon somehow
#### boolean switch

         IF(ln_co2sys) THEN
           WRITE(numout,*) 'am in p4z_flx walrus try to initialize co2sys'
         ENDIF

         IF( .NOT.ln_co2sys) THEN
           WRITE(numout,*) 'am in p4z_flx walrus stick with pisces'
         ENDIF

#### we probably don't need this jm loop, I think it refers to the interpolated co2
                 DO jj = 1, jpj
        !CDIR NOVERRCHK
                    DO ji = 1, jpi

                       ! DUMMY VARIABLES FOR DIC, H+, AND BORATE
    !                   zbot  = borat(ji,jj,1)
#### using total borate from SOG emp. fit - Uppstrom L 1974 
                        zbot  = tsn(ji,jj,1,jp_sal) * 0.0004157d0 / 35.0d0    ! in mol/kg-SW


#### grepping in p4z - still dunno what is rhop rtrn
#### QQ DD what is rhop, rtrn                    
    !                   zfact = rhop(ji,jj,1) / 1000. + rtrn
                        zfact = 1.0d0
                       zdic  = trb(ji,jj,1,jpdic) / zfact
#### QQ DD where and how is hi initialized? can I just initialize it as 1E-10?

#### grep: ./p4zsms.F90:               hi(ji,jj,jk) = ( ak23(ji,jj,jk) * zbicarb / zco3 ) * ztmas + 1.e-9 * ztmas1
#### ./p4zsms.F90:            CALL iom_get( numrtr, jpdom_autoglo, 'PH' , hi(:,:,:)  )
#### ./p4zsms.F90:!            hi(:,:,:) = 1.e-9

     !                  zph   = MAX( hi(ji,jj,1), 1.e-10 ) / zfact
                       zalka = trb(ji,jj,1,jpta) / zfact

                       ! CALCULATE [ALK]([CO3--], [HCO3-])

#### QQ DD akw3? aphscale? akb3? ak23? ak13? 

#### akw3, akb3, ak23, ak13, aphscale are all various chemistry constants that get calculated in a straightforward way in p4zche, may be easiest to just implement all of p4zche

     !                  zalk  = zalka - (  akw3(ji,jj,1) / zph - zph / aphscale(ji,jj,1)    &
      !                 &       + zbot / ( 1.+ zph / akb3(ji,jj,1) )  )

                       ! CALCULATE [H+] AND [H2CO3]
     !                  zah2   = SQRT(  (zdic-zalk)**2 + 4.* ( zalk * ak23(ji,jj,1)   &
      !                    &                                        / ak13(ji,jj,1) ) * ( 2.* zdic - zalk )  )
     !                  zah2   = 0.5 * ak13(ji,jj,1) / zalk * ( ( zdic - zalk ) + zah2 )
     !                  zh2co3(ji,jj) = ( 2.* zdic - zalk ) / ( 2.+ ak13(ji,jj,1) / zah2 ) * zfact
    !                   hi(ji,jj,1)   = zah2 * zfact
                    END DO
                 END DO




          ! --------------
          ! COMPUTE FLUXES
          ! --------------

          ! FIRST COMPUTE GAS EXCHANGE COEFFICIENTS
          ! -------------------------------------------

###### !QQ: does it make sense to output schmidt number and gas exchange?
##### this all works currently
    !CDIR NOVERRCHK
          DO jj = 1, jpj
    !CDIR NOVERRCHK
             DO ji = 1, jpi
                ztc  = MIN( 35., tsn(ji,jj,1,jp_tem) )
                ztc2 = ztc * ztc
                ztc3 = ztc * ztc2
                ztc4 = ztc2 * ztc2
    !~~~~~~~~~~~~~walrus F_B compute schmidt number~~~~~~~~~~~~~~~~~~
               ! Compute the schmidt Number both O2 and CO2
                zsch_co2 = 2116.8 - 136.25 * ztc + 4.7353 * ztc2 - 0.092307 * ztc3 + 0.0007555 * ztc4
                zsch_o2  = 1920.4 - 135.6  * ztc + 5.2122 * ztc2 - 0.109390 * ztc3 + 0.0009377 * ztc4
    !~~~~~~~~~~~~~walrus F_C compute piston velocity~~~~~~~~~~~~~~~~~
                !  wind speed
               zws = wndm(ji,jj) * wndm(ji, jj)

               ! Compute the piston velocity for O2 and CO2
                zkgwan = 0.251 * zws
                zkgwan = zkgwan * xconv * tmask(ji,jj,1)
###### !QQ: what is key_degrad?
    # if defined key_degrad
    ! not sure what this is
                zkgwan = zkgwan * facvol(ji,jj,1)
    #endif
    !~~~~~~~~~~~~~walrus F_D compute gas exchange~~~~~~~~~~~~~~~~~~~~
               ! compute gas exchange for CO2 and O2
               zkgco2(ji,jj) = zkgwan * SQRT( 660./ zsch_co2 )
               zkgo2 (ji,jj) = zkgwan * SQRT( 660./ zsch_o2 )

##### this is just me writing something out (gas exchange)
               oce_o2(ji,jj) = zkgo2(ji,jj)
               oce_co2(ji,jj) = zkgco2(ji,jj)
              !(ji,jj,1,jpcflx) = zkgco2(ji,jj)

             END DO
             WRITE(numout,*) 'am in the flux calculation loop walrus'
          END DO

          CALL iom_put("oce_co2",oce_co2)
          CALL iom_put("oce_o2",oce_o2)
    !~~~~~~~~~~~~~walrus F_E fluxes~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    !      DO jj = 1, jpj
    !         DO ji = 1, jpi
    !            ! Compute CO2 flux for the sea and air

#### chemo2? chemc? 
#### These are solubilities of o2 and co2, calculated in p4zche (which we don't have)

    !            zfld = satmco2(ji,jj) * patm(ji,jj) * tmask(ji,jj,1) * chemc(ji,jj) * zkgco2(ji,jj)   ! (mol/L) * (m/s)
    !            zflu = zh2co3(ji,jj) * tmask(ji,jj,1) * zkgco2(ji,jj)                                   ! (mol/L) (m/s) ?
    !            oce_co2(ji,jj) = ( zfld - zflu ) * rfact2 * e1e2t(ji,jj) * tmask(ji,jj,1) * 1000.
    !            ! compute the trend
    !            tra(ji,jj,1,jpdic) = tra(ji,jj,1,jpdic) + ( zfld - zflu ) * rfact2 / fse3t(ji,jj,1)

    !            ! Compute O2 flux
    !            zfld16 = patm(ji,jj) * chemo2(ji,jj,1) * tmask(ji,jj,1) * zkgo2(ji,jj)          ! (mol/L) * (m/s)
    !            zflu16 = trb(ji,jj,1,jpoxy) * tmask(ji,jj,1) * zkgo2(ji,jj)
    !            zoflx(ji,jj) = zfld16 - zflu16
    !            tra(ji,jj,1,jpoxy) = tra(ji,jj,1,jpoxy) + zoflx(ji,jj) * rfact2 / fse3t(ji,jj,1)
    !         END DO
    !      END DO

    WRITE(numout,*) 'walrus p4z_flx module entered'

       END SUBROUTINE p4z_flx

# ! Subroutine from PISCES, unchanged but annotated

###### ! In our code, I intend to take out everything between the !!!!!!!!!!!!!!START and !!!!!!!!!END lines


    SUBROUTINE p4z_flx ( kt, knt )
          INTEGER, INTENT(in) ::   kt, knt   !
          !
          INTEGER  ::   ji, jj, jm, iind, iindm1
          REAL(wp) ::   ztc, ztc2, ztc3, ztc4, zws, zkgwan
          REAL(wp) ::   zfld, zflu, zfld16, zflu16, zfact
          REAL(wp) ::   zph, zah2, zbot, zdic, zalk, zsch_o2, zalka, zsch_co2
          REAL(wp) ::   zyr_dec, zdco2dt
          CHARACTER (len=25) :: charout
          REAL(wp), POINTER, DIMENSION(:,:) :: zkgco2, zkgo2, zh2co3, zoflx, zw2d 
          !!---------------------------------------------------------------------
          !
          IF( nn_timing == 1 )  CALL timing_start('p4z_flx')
          !
          CALL wrk_alloc( jpi, jpj, zkgco2, zkgo2, zh2co3, zoflx )
          !

          ! SURFACE CHEMISTRY (PCO2 AND [H+] IN
          !     SURFACE LAYER); THE RESULT OF THIS CALCULATION
          !     IS USED TO COMPUTE AIR-SEA FLUX OF CO2

#### QQ: what is nit000? is p4z_patm called?

          IF( kt /= nit000 .AND. knt == 1 ) CALL p4z_patm( kt )    ! Get sea-level pressure (E&K [1981] climatology) for use in flux calcs

#### this a linear interpolaton we probably don't need (or definitely have to rewrite)

    !!!!!!!START
          IF( ln_co2int ) THEN 
             ! Linear temporal interpolation  of atmospheric pco2.  atcco2.txt has annual values.
             ! Caveats: First column of .txt must be in years, decimal  years preferably. 
             ! For nn_offset, if your model year is iyy, nn_offset=(years(1)-iyy) 
             ! then the first atmospheric CO2 record read is at years(1)
             zyr_dec = REAL( nyear + nn_offset, wp ) + REAL( nday_year, wp ) / REAL( nyear_len(1), wp )
             jm = 1
             DO WHILE( jm <= nmaxrec .AND. years(jm) < zyr_dec ) ;  jm = jm + 1 ;  END DO
             iind = jm  ;   iindm1 = jm - 1
             zdco2dt = ( atcco2h(iind) - atcco2h(iindm1) ) / ( years(iind) - years(iindm1) + rtrn )
             atcco2  = zdco2dt * ( zyr_dec - years(iindm1) ) + atcco2h(iindm1)
             satmco2(:,:) = atcco2 
          ENDIF
    !!!!!!!END      

#### QQ do we need this key? maybe we can just read in atm_co2 from somewhere or initialize it
    !!!!!!!START
    #if defined key_cpl_carbon_cycle
          satmco2(:,:) = atm_co2(:,:)
    #endif
    !!!! END

#### we probably don't need this jm loop, I think it refers to the interpolated co2
          DO jm = 1, 10
    !CDIR NOVERRCHK
             DO jj = 1, jpj
    !CDIR NOVERRCHK
                DO ji = 1, jpi

                   ! DUMMY VARIABLES FOR DIC, H+, AND BORATE
                   
#### in my code I calculate borate from salinity; this compiles and runs
                   zbot  = borat(ji,jj,1)
                   
#### grepping in p4z has not elucidated what the heck rhop or rtrn are, in my experience
#### QQ DD what is rhop, rtn


                   zfact = rhop(ji,jj,1) / 1000. + rtrn

                   zdic  = trb(ji,jj,1,jpdic) / zfact

#### QQ DD where and how is hi initialized? can I just initialize it as 1E-10?

                   zph   = MAX( hi(ji,jj,1), 1.e-10 ) / zfact
                   zalka = trb(ji,jj,1,jptal) / zfact

                   ! CALCULATE [ALK]([CO3--], [HCO3-])
                   
#### QQ DD akw3? aphscale? akb3? ak23? ak13? 

#### akw3, akb3, ak23, ak13, aphscale are all various chemistry constants that get calculated in a straightforward way in p4zche, may be easiest to just implement all of p4zche

                   zalk  = zalka - (  akw3(ji,jj,1) / zph - zph / aphscale(ji,jj,1)    &
                   &       + zbot / ( 1.+ zph / akb3(ji,jj,1) )  )

                   ! CALCULATE [H+] AND [H2CO3]
                   zah2   = SQRT(  (zdic-zalk)**2 + 4.* ( zalk * ak23(ji,jj,1)   &
                      &                                        / ak13(ji,jj,1) ) * ( 2.* zdic - zalk )  )
                   zah2   = 0.5 * ak13(ji,jj,1) / zalk * ( ( zdic - zalk ) + zah2 )
                   zh2co3(ji,jj) = ( 2.* zdic - zalk ) / ( 2.+ ak13(ji,jj,1) / zah2 ) * zfact
                   hi(ji,jj,1)   = zah2 * zfact
                END DO
             END DO
          END DO


          ! --------------
          ! COMPUTE FLUXES
          ! --------------

          ! FIRST COMPUTE GAS EXCHANGE COEFFICIENTS
          ! -------------------------------------------

###### !QQ: does it make sense to output schmidt number and gas exchange?
##### this all works currently
    !CDIR NOVERRCHK
          DO jj = 1, jpj
    !CDIR NOVERRCHK
             DO ji = 1, jpi
                ztc  = MIN( 35., tsn(ji,jj,1,jp_tem) )
                ztc2 = ztc * ztc
                ztc3 = ztc * ztc2 
                ztc4 = ztc2 * ztc2 
                ! Compute the schmidt Number both O2 and CO2
                zsch_co2 = 2116.8 - 136.25 * ztc + 4.7353 * ztc2 - 0.092307 * ztc3 + 0.0007555 * ztc4
                zsch_o2  = 1920.4 - 135.6  * ztc + 5.2122 * ztc2 - 0.109390 * ztc3 + 0.0009377 * ztc4
                !  wind speed 
                zws  = wndm(ji,jj) * wndm(ji,jj)
                ! Compute the piston velocity for O2 and CO2
                zkgwan = 0.251 * zws
                zkgwan = zkgwan * xconv * ( 1.- fr_i(ji,jj) ) * tmask(ji,jj,1)
                
###### !QQ: what is key_degrad?
    # if defined key_degrad
                zkgwan = zkgwan * facvol(ji,jj,1)
    #endif 
                ! compute gas exchange for CO2 and O2
                zkgco2(ji,jj) = zkgwan * SQRT( 660./ zsch_co2 )
                zkgo2 (ji,jj) = zkgwan * SQRT( 660./ zsch_o2 )
             END DO
          END DO

          DO jj = 1, jpj
             DO ji = 1, jpi
                ! Compute CO2 flux for the sea and air
                
#### chemo2? chemc? 
#### These are solubilities of o2 and co2, calculated in p4zche (which we don't have)

                zfld = satmco2(ji,jj) * patm(ji,jj) * tmask(ji,jj,1) * chemc(ji,jj) * zkgco2(ji,jj)   ! (mol/L) * (m/s)
                zflu = zh2co3(ji,jj) * tmask(ji,jj,1) * zkgco2(ji,jj)                                   ! (mol/L) (m/s) ?
                oce_co2(ji,jj) = ( zfld - zflu ) * rfact2 * e1e2t(ji,jj) * tmask(ji,jj,1) * 1000.
                ! compute the trend
                tra(ji,jj,1,jpdic) = tra(ji,jj,1,jpdic) + ( zfld - zflu ) * rfact2 / fse3t(ji,jj,1)

                ! Compute O2 flux 
                zfld16 = patm(ji,jj) * chemo2(ji,jj,1) * tmask(ji,jj,1) * zkgo2(ji,jj)          ! (mol/L) * (m/s)
                zflu16 = trb(ji,jj,1,jpoxy) * tmask(ji,jj,1) * zkgo2(ji,jj)
                zoflx(ji,jj) = zfld16 - zflu16
                tra(ji,jj,1,jpoxy) = tra(ji,jj,1,jpoxy) + zoflx(ji,jj) * rfact2 / fse3t(ji,jj,1)
             END DO
          END DO

####    !!!!!!!!!START
###### !global massive fluxes. in our code unnecessary g
          t_oce_co2_flx     = glob_sum( oce_co2(:,:) )                    !  Total Flux of Carbon
          t_oce_co2_flx_cum = t_oce_co2_flx_cum + t_oce_co2_flx       !  Cumulative Total Flux of Carbon
    !      t_atm_co2_flx     = glob_sum( satmco2(:,:) * e1e2t(:,:) )       ! Total atmospheric pCO2
          t_atm_co2_flx     =  atcco2      ! Total atmospheric pCO2

          IF(ln_ctl)   THEN  ! print mean trends (used for debugging)
             WRITE(charout, FMT="('flx ')")
             CALL prt_ctl_trc_info(charout)
             CALL prt_ctl_trc(tab4d=tra, mask=tmask, clinfo=ctrcnm)
          ENDIF

          IF( lk_iomput .AND. knt == nrdttrc ) THEN
             CALL wrk_alloc( jpi, jpj, zw2d )  
             IF( iom_use( "Cflx"  ) )  THEN
                zw2d(:,:) = oce_co2(:,:) / e1e2t(:,:) * rfact2r
                CALL iom_put( "Cflx"     , zw2d ) 
             ENDIF
             IF( iom_use( "Oflx"  ) )  THEN
                zw2d(:,:) =  zoflx(:,:) * 1000 * tmask(:,:,1)
                CALL iom_put( "Oflx" , zw2d )
             ENDIF
             IF( iom_use( "Kg"    ) )  THEN
                zw2d(:,:) =  zkgco2(:,:) * tmask(:,:,1)
                CALL iom_put( "Kg"   , zw2d )
             ENDIF
             IF( iom_use( "Dpco2" ) ) THEN
               zw2d(:,:) = ( satmco2(:,:) * patm(:,:) - zh2co3(:,:) / ( chemc(:,:) + rtrn ) ) * tmask(:,:,1)
               CALL iom_put( "Dpco2" ,  zw2d )
             ENDIF
             IF( iom_use( "Dpo2" ) )  THEN
               zw2d(:,:) = ( atcox * patm(:,:) - atcox * trn(:,:,1,jpoxy) / ( chemo2(:,:,1) + rtrn ) ) * tmask(:,:,1)
               CALL iom_put( "Dpo2"  , zw2d )
             ENDIF
             IF( iom_use( "tcflx" ) )  CALL iom_put( "tcflx"    , t_oce_co2_flx * rfact2r )   ! molC/s
             CALL iom_put( "tcflxcum" , t_oce_co2_flx_cum )      ! molC
             !
             CALL wrk_dealloc( jpi, jpj, zw2d )
          ELSE
             IF( ln_diatrc ) THEN
                trc2d(:,:,jp_pcs0_2d    ) = oce_co2(:,:) / e1e2t(:,:) * rfact2r 
                trc2d(:,:,jp_pcs0_2d + 1) = zoflx(:,:) * 1000 * tmask(:,:,1) 
                trc2d(:,:,jp_pcs0_2d + 2) = zkgco2(:,:) * tmask(:,:,1) 
                trc2d(:,:,jp_pcs0_2d + 3) = ( satmco2(:,:) * patm(:,:) - zh2co3(:,:) / ( chemc(:,:) + rtrn ) ) * tmask(:,:,1) 
             ENDIF
          ENDIF
          
####    !!!!!!!!!END

          !
          CALL wrk_dealloc( jpi, jpj, zkgco2, zkgo2, zh2co3, zoflx )
          !
          IF( nn_timing == 1 )  CALL timing_stop('p4z_flx')
          !
       END SUBROUTINE p4z_flx
       
   